<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/5.11.%20%E6%AE%8B%E5%B7%AE%E7%BD%91%E7%BB%9C(ResNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##5.11. 残差网络（ResNet）
让我们先思考一个问题：对神经网络模型添加新的层，充分训练后的模型是否只可能更有效地降低训练误差？理论上，原模型解的空间只是新模型解的空间的子空间。也就是说，如果我们能将新添加的层训练成恒等映射 f(x)=x ，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。针对这一问题，何恺明等人提出了残差网络（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。

###5.11.1. 残差块
让我们聚焦于神经网络局部。如图5.9所示，设输入为 $x$ 。假设我们希望学出的理想映射为 $f(x)$ ，从而作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射 $f(x)$ ，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射 $f(x)−x$ 。残差映射在实际中往往更容易优化。以本节开头提到的恒等映射作为我们希望学出的理想映射 $f(x)$ 。我们只需将图5.9中右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0，那么 $f(x)$ 即为恒等映射。实际中，当理想映射 $f(x)$ 极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。图5.9右图也是ResNet的基础块，即残差块（residual block）。在残差块中，输入可通过跨层的数据线路更快地向前传播。

<div align=center><img src="https://zh.gluon.ai/_images/residual-block.svg" width="500"/></div>

<center>图 5.9 设输入为 $x$ 。假设图中最上方激活函数输入的理想映射为 $f(x)$ 。左图虚线框中的部分需要直接拟合出该映射 $f(x)$ ，而右图虚线框中的部分需要拟合出有关恒等映射的残差映射 $f(x)−x$</center>

ResNet沿用了VGG全 $3\times 3$ 卷积层的设计。残差块里首先有2个有相同输出通道数的 $3\times 3$ 卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的 $1\times1$ 卷积层来将输入变换成需要的形状后再做相加运算。

残差块的实现如下。它可以设定输出通道数、是否使用额外的 $1\times1$ 卷积层来修改通道数以及卷积层的步幅。

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.data import Dataset
from tensorflow.keras import losses,optimizers
import time
import numpy as np
from tensorflow import image
import os
import sys

In [0]:
tf.enable_eager_execution()

In [0]:
class Residual(layers.Layer):
  def __init__(self,num_channels,use_1x1conv=False,strides=1):
    super(Residual,self).__init__()
    self.conv1=layers.Convolution2D(num_channels,kernel_size=3,padding='same',strides=strides)
    self.conv2=layers.Convolution2D(num_channels,kernel_size=3,padding='same')
    if use_1x1conv:
      self.conv3=layers.Convolution2D(num_channels,kernel_size=1,strides=strides)
    else:
      self.conv3=None
    self.bn1=layers.BatchNormalization()
    self.bn2=layers.BatchNormalization()

  def call(self,X):
    Y=keras.activations.relu(self.bn1(self.conv1(X)))
    Y=self.bn2(self.conv2(Y))
    if self.conv3:
      X=self.conv3(X)
    return keras.activations.relu(Y+X)

下面我们来查看输入和输出形状一致的情况。

In [0]:
blk=Residual(3)
X=tf.random.uniform(shape=(4,6,6,3))
blk(X).shape

TensorShape([Dimension(4), Dimension(6), Dimension(6), Dimension(3)])

我们也可以在增加输出通道数的同时减半输出的高和宽。

In [0]:
blk=Residual(6,use_1x1conv=True,strides=2)
blk(X).shape

TensorShape([Dimension(4), Dimension(3), Dimension(3), Dimension(6)])

###5.11.2. ResNet模型
ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的 $7\times7$ 卷积层后接步幅为2的 $3\times3$ 的最大池化层。不同之处在于ResNet每个卷积层后增加的批量归一化层。

In [0]:
net=Sequential([
  layers.Convolution2D(64,kernel_size=7,strides=2,padding='same'),
  layers.BatchNormalization(),
  layers.Activation('relu'),
  layers.MaxPool2D(pool_size=3,strides=2,padding='same')                
])

GoogLeNet在后面接了4个由Inception块组成的模块。ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。

In [0]:
def resnet_block(num_channels,num_residuals,first_block=False):
  blk=Sequential()
  for i in range(num_residuals):
    if i==0 and not first_block:
      blk.add(Residual(num_channels,use_1x1conv=True,strides=2))
    else:
      blk.add(Residual(num_channels))
  return blk

接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。

In [0]:
net.add(resnet_block(64,2,first_block=True))
net.add(resnet_block(128,2))
net.add(resnet_block(256,2))
net.add(resnet_block(512,2))

最后，与GoogLeNet一样，加入全局平均池化层后接上全连接层输出。

In [0]:
net.add(layers.GlobalAveragePooling2D())
net.add(layers.Dense(10))

这里每个模块里有4个卷积层（不计算 $1\times1$ 卷积层），加上最开始的卷积层和最后的全连接层，共计18层。这个模型通常也被称为ResNet-18。通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的ResNet-152。虽然ResNet的主体架构跟GoogLeNet的类似，但ResNet结构更简单，修改也更方便。这些因素都导致了ResNet迅速被广泛使用。

在训练ResNet之前，我们来观察一下输入形状在ResNet不同模块之间的变化。

In [0]:
X=tf.random.uniform(shape=(1,224,224,1))
for layer in net.layers:
  X=layer(X)
  print(layer.name,'output shape:\t',X.shape)

conv2d_5 output shape:	 (1, 112, 112, 64)
batch_normalization_4 output shape:	 (1, 112, 112, 64)
activation output shape:	 (1, 112, 112, 64)
max_pooling2d output shape:	 (1, 56, 56, 64)
sequential_1 output shape:	 (1, 56, 56, 64)
sequential_2 output shape:	 (1, 28, 28, 128)
sequential_3 output shape:	 (1, 14, 14, 256)
sequential_4 output shape:	 (1, 7, 7, 512)
global_average_pooling2d output shape:	 (1, 512)
dense output shape:	 (1, 10)


###5.11.3. 获取数据和训练模型
下面我们在Fashion-MNIST数据集上训练ResNet。

In [0]:
lr,num_epochs,batch_size=0.05,5,128
optimizer=optimizers.SGD(learning_rate=lr)
loss=losses.SparseCategoricalCrossentropy(from_logits=True)
buffer_size=1000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=x_train[:,:,:,np.newaxis]#将三维张量增加一个channel维
  x_test=x_test[:,:,:,np.newaxis]
  train_iter=Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  test_iter=Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  return train_iter,test_iter
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)

4423680/4422102 [==============================] - 0s 0us/step
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matchi

In [0]:
net.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
history=net.fit_generator(train_iter,validation_data=test_iter,epochs=num_epochs)

Epoch 1/5
469/469 [==============================] - 166s 354ms/step - loss: 0.5036 - acc: 0.7287 - val_loss: 1.3670 - val_acc: 0.5663
Epoch 2/5
469/469 [==============================] - 164s 350ms/step - loss: 0.2513 - acc: 0.9002 - val_loss: 0.2697 - val_acc: 0.9047
Epoch 3/5
469/469 [==============================] - 164s 350ms/step - loss: 0.1868 - acc: 0.9268 - val_loss: 0.3256 - val_acc: 0.8862
Epoch 4/5
469/469 [==============================] - 166s 354ms/step - loss: 0.1375 - acc: 0.9461 - val_loss: 0.2957 - val_acc: 0.9001
Epoch 5/5
469/469 [==============================] - 165s 352ms/step - loss: 0.0916 - acc: 0.9643 - val_loss: 0.2982 - val_acc: 0.9097


###5.11.4. 小结
* 残差块通过跨层的数据通道从而能够训练出有效的深度神经网络。
ResNet深刻影响了后来的深度神经网络的设计。